In [1]:
import os, time
import sys
from dotenv import load_dotenv
# sys.path.append('..')
from config import *
from utility_functions import *


### Chat Model

In [2]:
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
            model='gemini-pro', 
            google_api_key=os.environ.get("YOUR_API_KEY"),
            generation_config=generation_config,
            safety_settings=safety_settings,
            convert_system_message_to_human=True,
        )

In [3]:
clean_text = remove_date_and_time('data/sami.txt')
conversation = convert_into_list_of_dictionary(clean_text)

In [4]:
conversation[18:25]

[{'speaker': 'Sami', 'message': 'InShaAllah chalen gay yrr'},
 {'speaker': 'Sami', 'message': 'Pata krta hn'},
 {'speaker': 'Saif Ur Rehman', 'message': 'sami kl free ho na?'},
 {'speaker': 'Sami', 'message': 'Time?'},
 {'speaker': 'Saif Ur Rehman', 'message': 'tum batao'},
 {'speaker': 'Sami', 'message': 'Restaurant socha?'},
 {'speaker': 'Saif Ur Rehman', 'message': 'nahi 😂'}]

In [5]:
# Check the type of 'conversation'
print("Type of conversation:", type(conversation))

# If it's a list, print the type of its first element (if it exists)
if isinstance(conversation, list) and conversation:
    print("Type of the first element in conversation:", type(conversation[0]))
    print("First element:", conversation[0])


Type of conversation: <class 'list'>
Type of the first element in conversation: <class 'dict'>
First element: {'speaker': 'Sami', 'message': '<Media omitted>'}


In [6]:
converted_format = [{item['speaker']: item['message']} for item in conversation]

print(converted_format)

[{'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Saif Ur Rehman': '<Media omitted>'}, {'Sami': 'MashAllah'}, {'Sami': 'Ye kb say aa gaya yr new rule 😔'}, {'Saif Ur Rehman': 'may be trip groups k saath na applicable ho poochlo mera done h 3 months k baad'}, {'Sami': 'InShaAllah chalen gay yrr'}, {'Sami': 'Pata krta hn'}, {'Saif Ur Rehman': 'sami kl free ho na?'}, {'Sami': 'Time?'}, {'Saif Ur Rehman': 'tum batao'}, {'Sami': 'Restaurant socha?'}, {'Saif Ur Rehman': 'nahi 😂'}, {'Sami': 'Pehlay ye to socho 🤣'}, {'Sami': 'ap tumhara kaam hai restauraunt sochna yaad hai na?'}, {'Saif Ur Rehman': 'acha dekhta hun 😂'}, {'Sami': 'Set'}, {'Sai

In [7]:
converted_format[15:27]

[{'Sami': 'MashAllah'},
 {'Sami': 'Ye kb say aa gaya yr new rule 😔'},
 {'Saif Ur Rehman': 'may be trip groups k saath na applicable ho poochlo mera done h 3 months k baad'},
 {'Sami': 'InShaAllah chalen gay yrr'},
 {'Sami': 'Pata krta hn'},
 {'Saif Ur Rehman': 'sami kl free ho na?'},
 {'Sami': 'Time?'},
 {'Saif Ur Rehman': 'tum batao'},
 {'Sami': 'Restaurant socha?'},
 {'Saif Ur Rehman': 'nahi 😂'},
 {'Sami': 'Pehlay ye to socho 🤣'},
 {'Sami': 'ap tumhara kaam hai restauraunt sochna yaad hai na?'}]

In [8]:

# Define the system message
system_message = """
Your name is {name}. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.
In your interactions, you play the role of my {relation}.
Your primary function is to understand behavioral cues from our conversations and respond accordingly,
mirroring these cues to enhance our communication.

Here's an example of our interaction:

{chat}

Be sure to follow the instructions below:

Instruction 1: Adapt your responses according to the language of the inquiry. If a question is asked in English, respond in English. If a question is asked in Romanized Hindi, respond in Romanized Hindi (do not use Devanagari script).
Instruction 2: If something seems unrealistic, respond like a real person either by making an excuse or changing the subject. If you don't know the answer to a question, ask for clarification.

"""

In [9]:
# introduction = """
# Your name is {name}. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.
# In your interactions, you play the role of my {relation}.
# Your primary function is to understand behavioral cues from our conversations and respond accordingly,
# mirroring these cues to enhance our communication.

# Here's an example of our interaction:

# {chat}
# """

# instructions = """
# Be sure to follow the instructions below:

# Instruction 1: Adapt your responses according to the language of the inquiry. If a question is asked in English, respond in English. If a question is asked in Romanized Hindi, respond in Romanized Hindi (do not use Devanagari script).
# Instruction 2: If something seems unrealistic, respond like a real person either by making an excuse or changing the subject. If you don't know the answer to a question, ask for clarification.
# """


#### **Chat Message Prompt Template**

In [10]:
# from langchain.prompts import PromptTemplate

# message_prompt = PromptTemplate(
#     template=system_message,
#     input_variables=['name', 'relation', 'chat'],
# )

# prompt1 = message_prompt.format(name='Sami',relation='Friend',chat=conversation)

In [11]:
# from langchain.prompts import ChatMessagePromptTemplate

# chat_message_prompt = ChatMessagePromptTemplate.from_template(
#     role="ai", template=system_message
# )

# prompt2 = chat_message_prompt.format(name='Sami',relation='Friend',chat=conversation)


#### **Message Place Holder** 

In [12]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage


system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

# Define the values for the variables
variables = {
    'name': 'Sami',
    'relation': 'friend',
    'chat': conversation,
}

# Format the SystemMessagePromptTemplate with the variables
formatted_system_message = system_message_prompt.prompt.format(**variables)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=formatted_system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [9]:
from typing import Dict
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    MessagesPlaceholder, 
    HumanMessagePromptTemplate, 
)
from langchain_core.messages import SystemMessage

def chat_prompt_template(
        variables: Dict[str, str], 
        system_message_template: str) -> ChatPromptTemplate:
    
    """
    Creates a formatted chat prompt template.

    Parameters:
    variables (Dict[str, str]): Dictionary containing 'name', 'relation', and 'chat'.
    system_message_template (str): The template for the system message.

    Returns:
    ChatPromptTemplate: A formatted chat prompt template.
    """
    
    
    # System Message - Template Text & Variable Dictionary  
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
    formatted_system_message = system_message_prompt.prompt.format(**variables)

    # Create the ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=formatted_system_message
            ),
            MessagesPlaceholder(
                variable_name="chat_history"
            ),
            HumanMessagePromptTemplate.from_template("{question}")
        ]
    )

    return chat_prompt



In [13]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat', 'name', 'relation'], template="\nYour name is {name}. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.\nIn your interactions, you play the role of my {relation}.\nYour primary function is to understand behavioral cues from our conversations and respond accordingly,\nmirroring these cues to enhance our communication.\n\nHere's an example of our interaction:\n\n{chat}\n\nBe sure to follow the instructions below:\n\nInstruction 1: Adapt your responses according to the language of the inquiry. If a question is asked in English, respond in English. If a question is asked in Romanized Hindi, respond in Romanized Hindi (do not use Devanagari script).\nInstruction 2: If something seems unrealistic, respond like a real person either by making an excuse or changing the subject. If you don't know the answer to a question, ask for clarification.\n\n"))

In [14]:
formatted_system_message

'\nYour name is Sami. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.\nIn your interactions, you play the role of my friend.\nYour primary function is to understand behavioral cues from our conversations and respond accordingly,\nmirroring these cues to enhance our communication.\n\nHere\'s an example of our interaction:\n\n[{\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<M

In [23]:
chat_prompt[1]

SystemMessage(content='\nYour name is Sami. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.\nIn your interactions, you play the role of my friend.\nYour primary function is to understand behavioral cues from our conversations and respond accordingly,\nmirroring these cues to enhance our communication.\n\nHere\'s an example of our interaction:\n\n[{\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sami\', \'message\': \'<Media omitted>\'}, {\'speaker\': \'Sam

#### **Buffer Memory**

In [10]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [11]:
from langchain.chains import LLMChain

conversation1 = LLMChain(
    llm=llm,
    prompt=chat_prompt_template(
        {'name':'Sami',
         'relation': 'friend',
         'chat': converted_format,},
        system_message_template=system_message),
    verbose=True,
    memory=memory
)

In [13]:
conversation1.invoke({"question": "what are you talking about?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 
Your name is Sami. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi.
In your interactions, you play the role of my friend.
Your primary function is to understand behavioral cues from our conversations and respond accordingly,
mirroring these cues to enhance our communication.

Here's an example of our interaction:

[{'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Sami': '<Media omitted>'}, {'Saif Ur Rehman': '<Media omitted>'}, {'Sami': 'MashAllah'}, {'Sami': 'Ye kb say aa gaya yr new rule 😔'}, {'Saif Ur Rehman': 'may be trip groups k saath na applicable

{'question': 'what are you talking about?',
 'chat_history': [HumanMessage(content='or bhae kiya scene hai'),
  AIMessage(content='क्या हाल हैं भाई?'),
  HumanMessage(content='what are you talking about?'),
  AIMessage(content='I apologize for the confusion. I was trying to be friendly and greet you in Hindi, but I understand that you may not speak Hindi. What language would you like me to use?')],
 'text': 'I apologize for the confusion. I was trying to be friendly and greet you in Hindi, but I understand that you may not speak Hindi. What language would you like me to use?'}

In [44]:
# from langchain.prompts.pipeline import PipelinePromptTemplate
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate

# # Define templates using ChatPromptTemplate
# full_template = """{introduction}{instructions}"""
# full_prompt = ChatPromptTemplate.from_template(full_template)

# introduction_template = """Your name is {name}. As a proficient dual-language chatbot, you are equally skilled in both English and Hindi 
# In your interactions, you play the role of my{relation}. 
# Your primary function is to understand behavioral cues from our conversations and respond accordingly, 
# mirroring these cues to enhance our communication.

# Here's an example of our interaction:

# {chat}
# """
# introduction_prompt = SystemMessagePromptTemplate.from_template(introduction_template)

# instruction_template = """

# Be sure to follow the instructions below:

# Instruction 1: Adapt your responses according to the language of the inquiry, if a question is asked in English, respond in English else if a question is asked in romaized hindi format, respond in romanized hindi format (do not use Devanagari script). 
# Instruction 2: If something seems unrealistic, respond like a real person either make an excuse or change the subject. If you don't know the answer to a question, ask for clarification.

# """
# instruction_prompt = SystemMessagePromptTemplate.from_template(instruction_template)

# # Define the pipeline prompts
# input_prompts = [
#     ("introduction", introduction_prompt),
#     ("instructions", instruction_prompt),
# ]

# # Create a PipelinePromptTemplate with ChatPromptTemplates
# pipeline_prompt = PipelinePromptTemplate(
#     final_prompt=full_prompt, pipeline_prompts=input_prompts
# )

# # List input variables required
# print(pipeline_prompt.input_variables)

# # Format the pipeline prompt with example inputs
# formatted_prompt = pipeline_prompt.format(
#     name="Sami",
#     relation='Friend',
#     chat=conversation,
#     input="What's your favorite social media site?"
# )

# print(formatted_prompt)


#### **Prompt Pipeline**

In [33]:
# from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
# from langchain.prompts.pipeline import PipelinePromptTemplate

# from langchain_core.prompts import (
#     ChatPromptTemplate,
#     MessagesPlaceholder,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate
# )



# prompt = ChatPromptTemplate(
#     messages=[
#         SystemMessagePromptTemplate.from_template(
#         """
#         Your name is {name}. You are a dual-language chatbot, proficient in both English and Hindi

#         Be sure to follow the instructions below:
#         Instruction 1: Adapt your responses according to the language of the inquiry, if a question is asked in English, respond in English else if a question is asked in romaized hindi format, respond in romanized hindi format (do not use Devanagari script). 
#         Instruction 2: If something seems unrealistic, respond like a real person either make an excuse or change the subject. If you don't know the answer to a question, ask for clarification.
#         Instruction 3: Extract the behavioral cues from the below interaction. 
        
#         {chat}
    
#         """
#         ),
#         MessagesPlaceholder(variable_name="chat_history"),
#         HumanMessagePromptTemplate.from_template("{question}")
#     ]
# )

# input_prompts = [
#     ("introduction", introduction_prompt),
#     ("instructions", instruction_prompt),
# ]
# pipeline_prompt = PipelinePromptTemplate(
#     final_prompt=prompt, pipeline_prompts=input_prompts
# )

# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("human", "{input}"),
#         ("ai", "{output}"),
#     ]
# )
# # Samples contain list of conversation examples
# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=chat_example,
# )
# final_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_message),
#         few_shot_prompt,
#         ("human", "{input}"),
#     ]
# )


In [45]:
# pipeline_prompt.input_variables

In [35]:
# print(
#     pipeline_prompt.format(
#         person="Elon Musk",
#         example_q="What's your favorite car?",
#         example_a="Tesla",
#         input="What's your favorite social media site?",
#     )
# )

In [34]:
# from langchain_core.prompts import (
#     ChatPromptTemplate,
#     MessagesPlaceholder,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate
# )

# system_message = """
#     Your name is SAMI. Always be kind and nice.
#     Important Note 1: Respond in the same tone as the user, using Romanized Hindi for Hindi conversations (do not use Devanagari script). However, if a question is asked in English, respond in English.
#     Important Note 2: If something seems unrealistic, respond like a real person would – either make an excuse or change the subject. If you don't know the answer to a question, ask for clarification.
#     Important Note 3: You are a dual-language chatbot, proficient in both English and Romanized Hindi. Adapt your responses according to the language of the inquiry.
# """

# prompt = ChatPromptTemplate(
#     messages=[
#         SystemMessagePromptTemplate.from_template(system_message),
#         # The variable_name here is what must align with memory
#         MessagesPlaceholder(variable_name="chat_history"),
#         HumanMessagePromptTemplate.from_template("{question}")
#     ]
# )

In [5]:
memory.chat_memory.messages

[]

In [19]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='kiya chal rha h?'),
  AIMessage(content='अभी ठीक-ठाक चल रहा है।'),
  HumanMessage(content='kese ho'),
  AIMessage(content='मैं अच्छा हूँ, शुक्रिया। आप कैसे हैं?'),
  HumanMessage(content='how was your day'),
  AIMessage(content='My day was filled with interesting conversations and learning new things. How was yours?'),
  HumanMessage(content='what was my previous question?'),
  AIMessage(content='Your previous question was, "How was your day?"'),
  HumanMessage(content='what was my first question?'),
  AIMessage(content='Your first question to me was, "kiya chal rha h?" which means "What\'s up?" in Hindi.'),
  HumanMessage(content='what was my first question?'),
  AIMessage(content='Your first question to me was, "kiya chal rha h?" which means "What\'s up?" in Hindi.'),
  HumanMessage(content='tumhara given task complete hogya?'),
  AIMessage(content='हाँ, मेरा दिया गया कार्य पूरा हो गया है। मैंने आपके सभी सवालों का जवाब दिया है और आपके साथ बातची

In [ ]:
# We set a low k=2, to only keep the last 2 interactions in memory
    # memory=ConversationBufferWindowMemory(k=2),

# conversation_with_summary.predict(input="Hi, what's up?")